In [1]:
# Importing Python libraries
import json
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importing libraries for data preprocessing , cleaning
from sklearn import preprocessing
from collections import Counter
from sklearn.preprocessing import StandardScaler
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

In [4]:
# importing keras libraries for model building
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

Using TensorFlow backend.


In [5]:
# Reading the json records and appending in a list
tweets = []
for line in open('customersdata.json', 'r'):
    tweets.append(json.loads(line))

In [6]:
fraudulent=[]
customerEmail=[]
customerPhone=[]
customerDevice=[]
customerIPAddress=[]
customerBillingAddress=[]

orderId=[]
orderAmount=[]
orderState=[]
orderShippingAddress=[]

paymentMethodId=[]
paymentMethodRegistrationFailure=[]
paymentMethodType=[]
paymentMethodProvider=[]
paymentMethodIssuer=[]

transactionFailed=[]

In [7]:
# Iterating through the json records and fetching out data and then finally appending the values in particular list.
for dic1 in tweets:
    
    for i in range(len(dic1['orders'])):
        vl=list(dic1['customer'].values())
        frd=dic1['fraudulent']
        customerEmail.append(vl[0])
        customerPhone.append(vl[1])
        customerDevice.append(vl[2])
        customerIPAddress.append(vl[3])
        customerBillingAddress.append(vl[4])
        fraudulent.append(frd)

    for i in range(len(dic1['orders'])):
        dc=dic1['orders'][i]
        vl=list(dc.values())
        orderId.append(vl[0])
        orderAmount.append(vl[1])
        orderState.append(vl[2])
        orderShippingAddress.append(vl[3])

    for i in range(len(dic1['orders'])):
        if len(dic1['orders'])>len(dic1['paymentMethods']):
            dc=dic1['paymentMethods'][0]
            vl=list(dc.values())
            paymentMethodId.append(vl[0])
            paymentMethodRegistrationFailure.append(vl[1])
            paymentMethodType.append(vl[2])
            paymentMethodProvider.append(vl[3])
            paymentMethodIssuer.append(vl[4])
        else:
    
            dc=dic1['paymentMethods'][i]
            vl=list(dc.values())
            paymentMethodId.append(vl[0])
            paymentMethodRegistrationFailure.append(vl[1])
            paymentMethodType.append(vl[2])
            paymentMethodProvider.append(vl[3])
            paymentMethodIssuer.append(vl[4])

    for i in range(len(dic1['orders'])):
        dc=dic1['transactions'][i]
        vl=list(dc.values())
        transactionFailed.append(vl[4])





In [8]:
# 
df=pd.DataFrame()

In [9]:
df['fraudulent']=fraudulent
df['paymentMethodId']=paymentMethodId
df['paymentMethodIssuer']=paymentMethodIssuer
df['paymentMethodProvider']=paymentMethodProvider
df['paymentMethodRegistrationFailure']=paymentMethodRegistrationFailure
df['paymentMethodType']=paymentMethodType
df['orderId']=orderId
df['orderAmount']=orderAmount
df['orderShippingAddress']=orderShippingAddress
df['orderState']=orderState
df['transactionFailed']=transactionFailed

In [10]:
df

,fraudulent,paymentMethodId,paymentMethodIssuer,paymentMethodProvider,paymentMethodRegistrationFailure,paymentMethodType,orderId,orderAmount,orderShippingAddress,orderState,transactionFailed
0,False,wt07xm68b,Citizens First Banks,JCB 16 digit,True,card,vjbdvd,18,"5493 Jones Islands\nBrownside, CA 51896",pending,False
1,False,wt07xm68b,Citizens First Banks,JCB 16 digit,True,card,yp6x27,26,"5493 Jones Islands\nBrownside, CA 51896",fulfilled,False
2,True,y3xp697jx,Solace Banks,VISA 16 digit,True,bitcoin,nlghpa,45,"898 Henry Ports\nNew Keithview, CA 95893-2497",fulfilled,False
3,True,6krszxc05,Vertex Bancorp,VISA 16 digit,False,card,uw0eeb,23,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,fulfilled,False
4,True,5z1szj2he,His Majesty Bank Corp.,Diners Club / Carte Blanche,False,card,bn44oh,43,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",fulfilled,True
...,...,...,...,...,...,...,...,...,...,...,...
473,False,f0xnu5om9,His Majesty Bank Corp.,JCB 16 digit,False,card,ow1cih,23,"130 Kimberly Junctions\nEricmouth, KS 59756-2919",fulfilled,False
474,False,f0xnu5om9,His Majesty Bank Corp.,JCB 16 digit,False,card,ak5a9n,32,"750 Sarah Stream\nAndersonfurt, WI 68970",fulfilled,False
475,False,f0xnu5om9,His Majesty Bank Corp.,JCB 16 digit,False,card,1bubxa,25,"130 Kimberly Junctions\nEricmouth, KS 59756-2919",fulfilled,True
476,False,f0xnu5om9,His Majesty Bank Corp.,JCB 16 digit,False,card,p0gdbf,19,"130 Kimberly Junctions\nEricmouth, KS 59756-2919",fulfilled,True


In [43]:
# saving the parsed json data in csv
df.to_csv('customer_parsed_data.csv')

In [12]:
# Defining function for one hot encoding categorcal columns.
from sklearn.preprocessing import LabelBinarizer
dat=[]
def catego_enco(df,col):
    values=np.array(df[col].tolist())
    label_encoder = preprocessing.LabelEncoder() 
    #label_encoder.fit(data[col])
    int_enco=label_encoder.fit_transform(values)
    
    
    
    onehotencoder = OneHotEncoder(sparse=False) 
    int_enco=int_enco.reshape(len(int_enco),1)
    onehot_encoded=onehotencoder.fit_transform(int_enco)
    df4=pd.DataFrame(onehot_encoded).add_prefix(col+'_')
    dat.append(df4)

    
    return(label_encoder,onehotencoder)

In [13]:
# Defining function for label encoding categorical variables
def Label_enco(df,col):
    values=np.array(df[col].tolist())
    label_encoder3 = preprocessing.LabelEncoder() 
    #label_encoder.fit(data[col])
    int_enco=label_encoder3.fit_transform(values)
    #print(int_enco)
    return(int_enco,label_encoder3)

In [14]:
# scalling the dataset for better performance of model and reducing complexity
from sklearn.preprocessing import StandardScaler
def numeric_scaller(df,col):
    scaller=StandardScaler()
    scalled=scaller.fit_transform(df[col].values.reshape(-1,1))
    return(scaller,scalled)

In [15]:
df.columns

Index(['fraudulent', 'paymentMethodId', 'paymentMethodIssuer',
       'paymentMethodProvider', 'paymentMethodRegistrationFailure',
       'paymentMethodType', 'orderId', 'orderAmount', 'orderShippingAddress',
       'orderState', 'transactionFailed'],
      dtype='object')

In [16]:
#binary_categorical_cols=['fraudulent',  'paymentMethodRegistrationFailure','orderState', 'transactionFailed']

In [17]:
binary_categorical_cols=['fraudulent', 'paymentMethodId', 'paymentMethodIssuer',
       'paymentMethodProvider', 'paymentMethodRegistrationFailure',
       'paymentMethodType', 'orderId', 'orderShippingAddress',
       'orderState', 'transactionFailed']

In [18]:
num_scaller_ins=[]
c,scalled_OrderAmount=numeric_scaller(df,'orderAmount')
num_scaller_ins.append(c)

In [19]:
binary_encoded_cols=[]
binary_ecoder_ins=[]
for i in binary_categorical_cols:
    binary_ecoded,be=Label_enco(df,i)
    binary_encoded_cols.append(binary_ecoded)
    binary_ecoder_ins.append(be)
    

In [20]:
len(binary_encoded_cols)

10

In [21]:
dat

[]

In [22]:
df2 = pd.DataFrame()
#ind=data.index
#df.set_index(ind,inplace=True)

In [23]:
df2['fraudulent']=binary_encoded_cols[0]
df2['paymentMethodId']=binary_encoded_cols[1]
df2['paymentMethodIssuer']=binary_encoded_cols[1]
df2['paymentMethodProvider']=binary_encoded_cols[1]
df2['paymentMethodRegistrationFailure']=binary_encoded_cols[1]
df2['paymentMethodType']=binary_encoded_cols[1]
df2['orderId']=binary_encoded_cols[1]
df2['orderShippingAddress']=binary_encoded_cols[1]
df2['orderState']=binary_encoded_cols[1]
df2['transactionFailed']=binary_encoded_cols[1]
df2['orderAmount']=scalled_OrderAmount

In [24]:
df2

,fraudulent,paymentMethodId,paymentMethodIssuer,paymentMethodProvider,paymentMethodRegistrationFailure,paymentMethodType,orderId,orderShippingAddress,orderState,transactionFailed,orderAmount
0,0,163,163,163,163,163,163,163,163,163,-0.813444
1,0,163,163,163,163,163,163,163,163,163,-0.443837
2,1,170,170,170,170,170,170,170,170,170,0.433979
3,1,39,39,39,39,39,39,39,39,39,-0.582440
4,1,36,36,36,36,36,36,36,36,36,0.341577
...,...,...,...,...,...,...,...,...,...,...,...
473,0,77,77,77,77,77,77,77,77,77,-0.582440
474,0,77,77,77,77,77,77,77,77,77,-0.166632
475,0,77,77,77,77,77,77,77,77,77,-0.490038
476,0,77,77,77,77,77,77,77,77,77,-0.767243


In [25]:
data2=df2.copy()

In [26]:
# poping out target column
target=df2.pop('fraudulent')

In [27]:
# Finding feature importance scores
from sklearn.ensemble import ExtraTreesClassifier as etc
x=data2.values
y=target.values
model=etc()
model.fit(x,y)
score=model.feature_importances_
score

array([0.8893941 , 0.01106907, 0.00676389, 0.00806703, 0.01053394,
       0.00861144, 0.00851763, 0.01050858, 0.01306313, 0.02216026,
       0.0113109 ])

In [28]:
#appending feature names with their scores 
feature_importance=list(zip(df2.columns,score))
from operator import itemgetter
feature_importance=sorted(feature_importance,key=itemgetter(1),reverse=True)


In [29]:
# Important Features with their scores
feature_importance

[('paymentMethodId', 0.8893941039434156),
 ('orderAmount', 0.022160261789247116),
 ('transactionFailed', 0.013063134435509582),
 ('paymentMethodIssuer', 0.011069074142107439),
 ('paymentMethodType', 0.010533944231657596),
 ('orderState', 0.010508581979268865),
 ('orderId', 0.008611444374888023),
 ('orderShippingAddress', 0.008517634484442666),
 ('paymentMethodRegistrationFailure', 0.008067030669556146),
 ('paymentMethodProvider', 0.006763893869680673)]

In [30]:
# spliting dataset for train and test
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test= train_test_split(df2,target, test_size=0.2)

In [31]:
model = Sequential()
model.add(Dense(10, input_shape=(10,), activation="relu"))
model.add(Dense(7, activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [32]:
model.fit(x_train,y_train,batch_size=20,epochs=50)


Epoch 1/50
382/382 [==============================] - 0s 844us/step - loss: 2.2277 - acc: 0.4974
Epoch 2/50
382/382 [==============================] - 0s 78us/step - loss: 1.0126 - acc: 0.4503
Epoch 3/50
382/382 [==============================] - 0s 65us/step - loss: 0.7698 - acc: 0.4686
Epoch 4/50
382/382 [==============================] - 0s 81us/step - loss: 0.7276 - acc: 0.4607
Epoch 5/50
382/382 [==============================] - 0s 70us/step - loss: 0.7662 - acc: 0.5183
Epoch 6/50
382/382 [==============================] - 0s 76us/step - loss: 0.7526 - acc: 0.4895
Epoch 7/50
382/382 [==============================] - 0s 86us/step - loss: 0.7568 - acc: 0.4634
Epoch 8/50
382/382 [==============================] - 0s 91us/step - loss: 0.7962 - acc: 0.4581
Epoch 9/50
382/382 [==============================] - 0s 86us/step - loss: 0.7154 - acc: 0.5000
Epoch 10/50
382/382 [==============================] - 0s 73us/step - loss: 0.7486 - acc: 0.5445
Epoch 11/50
382/382 [================

In [33]:
# Predicting test data (unseen data)
predicted=model.predict(x_test)

In [34]:
pred=[]
for i in predicted:
    if i>=0.5:
        pred.append(1)
    else:
        pred.append(0)

In [35]:
# applying confusion matrix 
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, pred)
print(conf_mat)

[[21 35]
 [17 23]]


In [36]:
# finding the  proportion of actual positives was identified correctly
recall=conf_mat[0][0]/(conf_mat[0][0]+conf_mat[1][0])

In [37]:
# finding the proportion of positive identifications was actually correct
precision=conf_mat[0][0]/(conf_mat[0][0]+conf_mat[0][1])

In [38]:
precision

0.375

In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.55      0.38      0.45        56
           1       0.40      0.57      0.47        40

    accuracy                           0.46        96
   macro avg       0.47      0.47      0.46        96
weighted avg       0.49      0.46      0.46        96



In [40]:
Accuracy=(conf_mat[0][0]+conf_mat[1][1])/(conf_mat[0][0]+conf_mat[0][1]+conf_mat[1][0]+conf_mat[1][1])

In [42]:
#list of predicted value for test data
predic=[]
for i in pred:
    if i==0:
        predic.append('False')
    else:
        predic.append('True')
predic

['False',
 'True',
 'True',
 'False',
 'False',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'False',
 'False',
 'False',
 'True',
 'True',
 'False',
 'False',
 'True',
 'True',
 'True',
 'False',
 'True',
 'False',
 'True',
 'False',
 'False',
 'False',
 'True',
 'False',
 'True',
 'False',
 'True',
 'False',
 'True',
 'True',
 'False',
 'False',
 'True',
 'False',
 'False',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'False',
 'True',
 'False',
 'True',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'False',
 'False',
 'True',
 'False',
 'False',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'True',
 'False',
 'True',
 'True',
 'True',
 'False',
 'False',
 'False',
 'True',
 'True',
 'False',
 'False',
 'False',
 'True']